In [5]:
# Merging 12 months of sale data to a single file

import os
import pandas as pd

files = [file for file in os.listdir('C:\\Python Learning\\Pandas-Data-Science-Tasks\\SalesAnalysis\\Sales_Data')]

all_months_data = pd.DataFrame()

for f in files:    
    file = os.path.join('C:\\Python Learning\\Pandas-Data-Science-Tasks\\SalesAnalysis\\Sales_Data',f)
    df = pd.read_csv(file)
    all_months_data = pd.concat([all_months_data,df])

all_months_data.to_csv('all_data.csv',index=False)


# Read data from new dataframe
all_data = pd.read_csv('all_data.csv')


# Search & Drop rows of NaN
nan_df = all_data[all_data.isna().any(axis=1)]
all_data = all_data.dropna(how='all')


# Find 'Or' and delete it
all_data = all_data[all_data['Order Date'].str[0:2] != 'Or']


# Add Month column
all_data['Month'] = all_data['Order Date'].str[0:2]
all_data['Month'] = all_data['Month'].astype('int32')


# Convert columns(string) to the correct type(int)
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])


# Add 'Sales' column
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']

In [6]:
# Question 1: What was the best month for sales? How much was earned that month?
monthly_sale = all_data.groupby('Month').sum()['Sales']
best_month = monthly_sale.idxmax()  
best_month_sales = monthly_sale.max() 

In [7]:
# Question 2: What city sold the most product?
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

# Other way above:  all_data['City'] = all_data['Purchase Address'].apply(get_city)
all_data['City'] = all_data['Purchase Address'].apply(lambda x:f"{get_city(x)} ({get_state(x)})")
city_sale = all_data.groupby('City').sum()['Sales']
best_city = city_sale.idxmax()

In [8]:
# Question 3 : What time should we display adverisements to maximize likelihood of customers's buying product?
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Minute'] = all_data['Order Date'].dt.minute

C:\Users\df-admin\AppData\Local\Temp\ipykernel_15608\1627820695.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])


In [ ]:
# Question 4 : What products are most often sold together?
df = all_data[all_data['Order ID'].duplicated(keep=False)]
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))     # ***awesome***
df = df[['Order ID','Grouped']].drop_duplicates()

from itertools import combinations
from collections import Counter
count = Counter()
for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))

for key,value in count.most_common(10):
    print(key,value)

In [ ]:
# Question 5 : What product sold the most? Why do you think it sold the most?
product_group = all_data.groupby('Product')
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'], errors='coerce')
qty_group = product_group['Quantity Ordered'].sum()
most_product = qty_group.idxmax()
print(most_product)

AAA Batteries (4-pack)


In [4]:
from functools import reduce
nums = [10,20,22,25,29,35]
sum = reduce(lambda x,y : x+y, nums)

max = reduce(lambda x,y : x if x > y else y , nums)

scores = [[1,35,80],[2,32,75],[3,30,82],[4,33,75],[5,37,60]]
avg = 35
new_marks = map(lambda x : x[2]+2 if x[1] >= avg else x[2]-2, scores )
list(new_marks)

[82, 73, 80, 73, 62]